In [ ]:
import numpy as np
import cv2,os

import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist
from skimage.measure import label, regionprops

%matplotlib inline

In [ ]:
dataNum = 0
dirNames = ["cfdn-f11/","cfdn-f22/","ox-1s/"]
rawImgPaths = [
    "/media/localuser/Windows/Users/Default/Desktop/datasets/cfdn-f11/",
    "/media/localuser/Windows/Users/Default/Desktop/datasets/cfdn-f22/",
    "../../../datasets/oxford/1-s/"
              ]

dataLoadPath = "../out/" + dirNames[dataNum]
rawImgPath = rawImgPaths[dataNum]
outDir = "../out/filtered-loop-track/" + dirNames[dataNum]

if not os.path.exists(outDir):
    os.makedirs(outDir)
    
imgCount = len(os.listdir(rawImgPath))

fileExtRaw = ".jpg"
if dataNum in [2]:
    fileExtRaw = ".png"
    
    
f0, xx0, yy0 = 983.044006, 643.646973, 493.378998
f0, xx0, yy0 = 0.8*f0, 0.8*xx0, 0.8*yy0

In [ ]:
def loadProcImg(dataPath,idx,emb=False,colLab=False):
    img1 = cv2.imread(dataPath+"{0:07d}".format(idx)+fileExtRaw)
    
    binImg = img1[:,2*img1.shape[1]//4:3*img1.shape[1]//4]
    embImg = img1[:,img1.shape[1]//4:2*img1.shape[1]//4]
    colLabImg = img1[:,3*img1.shape[1]//4:]
    
    if emb and colLab:
        return binImg, embImg, colLabImg
    elif emb:
        return binImg, embImg
    elif colLab:
        return binImg, colLabImg
    else:
        return binImg

def loadRawImg(dataPath,idx):
    img1 = cv2.imread(dataPath+"{0:07d}".format(idx)+fileExtRaw)
    return img1#cv2.resize(img1,(512,256),interpolation=cv2.INTER_LINEAR)

def getLanePoints(idxOrImg,passedImg=False):
    if passedImg:
        im = idxOrImg.copy()        
    else:
        im = loadProcImg(dataLoadPath,idxOrImg)
    lanePts = np.where(im[:,:,0]==255)
    return np.array(lanePts).transpose()
    
tstIdx = 2
lanePts1 = getLanePoints(tstIdx)
plt.imshow(loadRawImg(rawImgPath,tstIdx))
plt.plot(lanePts1[:,1],lanePts1[:,0])

In [ ]:
# def getLabelsFromBinaryImg(inImg):
#     imL = label(inImg)
#     regData = regionprops(imL)
#     for i1 in range(len(regions)):
#         ...
#     # to do

def getLanesFromPoints(inPts,inImg):
    
    if inPts is None or inPts.shape[0] == 0:
        return None, None
    
    binImg = np.zeros(inImg.shape,np.uint8)
    binImg[inPts[:,0],inPts[:,1],:] = (255,255,255)

    procImg = cv2.dilate(binImg,np.ones((21,21),np.uint8))
    
    labImg = label(procImg)

    regions = regionprops(labImg)
    
    laneImg = np.zeros(inImg.shape,np.uint8)
    lanePointsAll = []
    for i1 in range(len(regions)):
        laneImgReg = np.zeros(inImg.shape,np.uint8)
        cords = regions[i1].coords
        y = cords[:,0]
        x = cords[:,1]

        fit = np.polyfit(x,y,1)
        fit_fn = np.poly1d(fit) 

        xnew = np.arange(x.min(),x.max())
        ynew = fit_fn(xnew).astype(int)

#         laneImg[ynew,xnew,:] = (255,255,255)
        laneImgReg = cv2.line(laneImgReg,(xnew[0],int(fit_fn(xnew[0]))),(xnew[-1],int(fit_fn(xnew[-1]))),
                          (255,255,255),2,cv2.LINE_AA)
    
        laneImg = cv2.line(laneImg,(xnew[0],int(fit_fn(xnew[0]))),(xnew[-1],int(fit_fn(xnew[-1]))),
                          (255,255,255),2,cv2.LINE_AA)
        
        lanePtsReg = np.array(np.where(laneImgReg[:,:,0]==255)).transpose()
        print(lanePtsReg.shape)
        
        lanePointsAll.append(lanePtsReg)
        
    lanePointsAll = np.concatenate(lanePointsAll)
    return laneImg, lanePointsAll

# idx = 7
# img1 = loadRawImg(rawImgPath,idx)
# prevPoints = getProjectedPoints(idx,idx+1)
# print(prevPoints.shape) 

# laneImg, lanePoints = getLanesFromPoints(prevPoints,img1)
# plt.imshow(laneImg)

In [ ]:
colorsDefLoc = np.array([np.array([255, 0, 0]),
                           np.array([0, 255, 0]),
                           np.array([0, 0, 255]),
                           np.array([255, 255, 0]),
                           np.array([0, 255, 255]),
                           np.array([255, 0, 255]),
                           np.array([255, 255, 255]),
#                            np.array([125, 125, 125])
                        ])
    
def getLabelFromEmbedding(embImg):

    im2 = np.argmin(np.sum(abs(embImg[None] - colorsDefLoc[:,None,None,:]),axis=-1),axis=0).astype(np.uint8)
    return im2


def verifyPointEmbedding(pts,inEmbImg):
    embImg = inEmbImg.copy()
    embImg[:,:,2] = 0 # last channel is background
    embImg = np.max(embImg,axis=2) # channel color doesn't matter here
    embImg[embImg<embImg.mean()+2*embImg.std()] = 0 # some thresholding to get good embeddings (rest set to 0)
#     plt.imshow(embImg)
#     plt.colorbar()
#     plt.show()

    mutualPtsIdx = np.argwhere(embImg[pts[:,0],pts[:,1]] > 0)[:,0] # keep only those points that have +ve embeddings
    
    return mutualPtsIdx
def getLanesFromLabelledPoints(inPts,inLabels,inImg):
    uniLabels = np.unique(inLabels)
    laneImg = np.zeros(inImg.shape,np.uint8)
    lanePointsAll = []
    for lab in uniLabels:
        if lab==2:
            continue
        col1 = colorsDefLoc[lab]
        laneImgReg = np.zeros(inImg.shape,np.uint8)

        inds = np.where(inLabels==lab)
        cords = inPts[inds]

        y = cords[:,0]
        x = cords[:,1]
                
#         if len(cords) < 100:
#             continue

        fit = np.polyfit(x,y,1)
        fit_fn = np.poly1d(fit) 

        xnew = np.arange(x.min(),x.max())
        ynew = fit_fn(xnew).astype(int)
    
        if xnew.shape[0] == 0:
            continue
        
        laneImgReg = cv2.line(laneImgReg,(xnew[0],int(fit_fn(xnew[0]))),(xnew[-1],int(fit_fn(xnew[-1]))),
                          (255,255,255),2,cv2.LINE_AA)
        
        lanePtsReg = np.array(np.where(laneImgReg[:,:,0]==255)).transpose()
        
        goodInds = verifyPointEmbedding(lanePtsReg,inImg)
        if len(goodInds) < 0.5*len(lanePtsReg):
            continue
        
        lanePointsAll.append(lanePtsReg)
        laneImg = cv2.line(laneImg,(xnew[0],int(fit_fn(xnew[0]))),(xnew[-1],int(fit_fn(xnew[-1]))),
                          (int(col1[0]),int(col1[1]),int(col1[2])),2,cv2.LINE_AA)
        
    if len(lanePointsAll) != 0:   
        lanePointsAll = np.concatenate(lanePointsAll)
    else:
        lanePointsAll = None
#     newLanePts = np.array(np.where(laneImg==255)).transpose()
    
    return laneImg, lanePointsAll

In [ ]:
def trackPoints_opFlow(imRaw1,imRaw2,inLanePts1):
    
    imC1 = imRaw1#[300:500]
    imC2 = imRaw2#[300:500]
    
    flowImg = cv2.calcOpticalFlowFarneback(cv2.cvtColor(imC1,cv2.COLOR_BGR2GRAY),
                             cv2.cvtColor(imC2,cv2.COLOR_BGR2GRAY),
                             None,0.5,3,11,100,5,1.1,cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    
    inPts1 = inLanePts1.astype(int).copy()
#     inPts1 = inLanePts1 - [300,0]
#     print(inPts1)
    
    lp_flow = flowImg[inPts1[:,0],inPts1[:,1]]
    outPts2 = inPts1 + np.fliplr(lp_flow)

#     plt.figure(figsize=(15,10))
#     showImg=getCorresImg(imC1,imC2,list(zip(inPts1,outPts2)),True,1)    
#     plt.show()

#     return inPts1 + [300,0], outPts2 + [300,0]
    return outPts2

def trackPoints_readOpFlowData(imIdx,inLanePts1):
    flowImg = cv2.imread("../../UnFlow/out/CSS/"+"{0:06d}".format(imIdx)+"_flow.png",
                     cv2.IMREAD_UNCHANGED)[:,:,:2].astype(float)-500
    
    inPts1 = inLanePts1.astype(int).copy()
#     inPts1 = inLanePts1 - [300,0]
#     print(inPts1)
    
    lp_flow = flowImg[inPts1[:,0],inPts1[:,1]]
    outPts2 = inPts1 + np.fliplr(lp_flow)
    
    return outPts2

def getUnion2D(ptSet1,ptSet2):
    disMat = cdist(ptSet1,ptSet2)
    numNonZeroCols = np.count_nonzero(disMat,axis=0)
    inds = np.argwhere(numNonZeroCols==disMat.shape[0])[:,0]
    return inds#np.concatenate([ptSet1,ptSet2[inds]])
    
def getProjectedPoints(idx1,idx2,prevPoints=None,prevLabels=None):

    im1 = loadRawImg(rawImgPath,idx1)
    im2 = loadRawImg(rawImgPath,idx2)
    im1B,im1E,im1L = loadProcImg(dataLoadPath,idx1,True,True) 
    im2B,im2E,im2L = loadProcImg(dataLoadPath,idx2,True,True) 

    lanePts1 = getLanePoints(im1B,passedImg=True)
    labelImg1 = getLabelFromEmbedding(im1E)
    labels1 = labelImg1[lanePts1[:,0],lanePts1[:,1]]
    
    lanePts2 = getLanePoints(im2B,passedImg=True)
    labelImg2 = getLabelFromEmbedding(im2E)
        
    if prevPoints is not None:
        unionInds = getUnion2D(lanePts1,prevPoints)
        lanePts1 = np.concatenate([lanePts1,prevPoints[unionInds]])
        labels1 = np.concatenate([labels1,prevLabels[unionInds]])
        
#     plt.figure(figsize=(15,10))
#     plt.imshow(cv2.cvtColor(im2,cv2.COLOR_BGR2RGB),)
    displayImg = im2.copy()
    laneImg = np.zeros(displayImg.shape,displayImg.dtype)
    allPointImg = laneImg.copy()
    print(lanePts1.shape)
    fittedPoints, fittedLabels = None, None
    if lanePts1.shape[0] != 0:
    
    #     trackedPts2 = trackPoints_opFlow(im1,im2,lanePts1)
        trackedPts2 = trackPoints_readOpFlowData(idx1,lanePts1)
        trackedPts2 = np.floor(trackedPts2).astype(int)

        boundedInds = np.where((trackedPts2<im2E.shape[:2]).prod(axis=1)==1)[0] # point indices that are beyond image boundary
        trackedPts2 = trackedPts2[boundedInds]
    
        intersectInds = verifyPointEmbedding(trackedPts2,im2E)
        mutualPts2 = trackedPts2[intersectInds].astype(int)
        labels2_ =  labelImg2[mutualPts2[:,0],mutualPts2[:,1]]

        allPointImg[mutualPts2[:,0],mutualPts2[:,1]] = colorsDefLoc[labels2_]#[255,255,255]
        laneImg, fittedPoints = getLanesFromLabelledPoints(mutualPts2,labels2_,im2E)#getLanesFromPoints(mutualPts2,im2)
        if fittedPoints is not None:
            fittedLabels = labelImg2[fittedPoints[:,0],fittedPoints[:,1]]
        
##         plt.scatter(mutualPts2[:,1],mutualPts2[:,0],c=mutualPtsColor/255.0)
#         plt.plot(fittedPoints[:,1],fittedPoints[:,0],'g.')  
            displayImg[fittedPoints[:,0],fittedPoints[:,1]] = [0,255,0]
     
    displayImg[lanePts2[:,0],lanePts2[:,1]] = [0,0,255]
        
#     plt.plot(lanePts2[:,1],lanePts2[:,0],'r.')

#     plt.gca().axis("off")    
#     plt.tight_layout()
#     plt.savefig(outDir+str(idx2)+".jpg",bbox_inches='tight', pad_inches = 0)
#     plt.close()  
    cv2.imwrite(outDir+str(idx2)+".jpg",np.hstack([displayImg,allPointImg,laneImg]))
    
    return fittedPoints, fittedLabels

In [ ]:
# idx1, idx2 = 15,16
# pts2 = getProjectedPoints(idx1,idx2)

prevPoints = getLanePoints(0,passedImg=False)
im1B,im1E,im1L = loadProcImg(dataLoadPath,0,True,True) 
labelImg = getLabelFromEmbedding(im1E)
prevLabels = labelImg[prevPoints[:,0],prevPoints[:,1]]
    
for i1 in range(imgCount-1):
    print(i1)
    prevPoints, prevLabels = getProjectedPoints(i1,i1+1,prevPoints,prevLabels)


#### Compare output 

In [ ]:
outDirComp = outDir + "compare/"
if not os.path.exists(outDirComp):
    os.makedirs(outDirComp)
    
for i1 in range(imgCount):
    
    im1 = loadRawImg(rawImgPath,i1)
    im1B = loadProcImg(dataLoadPath,i1) 
    lanePts1 = getLanePoints(im1B,passedImg=True)
    
    plt.figure(figsize=(15,10))
    plt.imshow(cv2.cvtColor(im1,cv2.COLOR_BGR2RGB),)
    plt.plot(lanePts1[:,1],lanePts1[:,0],'r.')
    
    plt.gca().axis("off")    
    plt.tight_layout()
    plt.savefig("./tmp.jpg",bbox_inches='tight', pad_inches = 0)
    plt.close()     
    
    imgTmp = cv2.imread("./tmp.jpg")
    imgNew = cv2.imread(outDir+str(i1)+".jpg")
    if imgNew is None:
        imgNew = imgTmp
        
    imgFinal = np.concatenate([imgTmp,imgNew],axis=0)
    
    cv2.imwrite(outDirComp+str(i1)+".jpg",imgFinal)
    
    

In [ ]:
# track within a pair

def getCorresImg(img1,img2,matchingCords,show=False,circOrRec=0,scaleFac=1):
    
    color = plt.cm.rainbow(np.linspace(0,1,len(matchingCords)))
    outImg = np.concatenate([img1,img2],axis=1)
    for i in range(len(matchingCords)):
        ptLoc1 = (matchingCords[i][0]).astype(int)
        ptLoc2 = (matchingCords[i][1]).astype(int)
        
        ptLoc2Shifted = ptLoc2.copy()
        ptLoc2Shifted[1] += img1.shape[1]

        if circOrRec == 0:
            cv2.circle(outImg,(ptLoc1[1],ptLoc1[0]),1,(255,255,255),1,cv2.LINE_AA)
            cv2.circle(outImg,(ptLoc2Shifted[1],ptLoc2Shifted[0]),1,(255,255,255),1,cv2.LINE_AA)
        else:
            cv2.rectangle(outImg,(ptLoc1[1]-scaleFac,ptLoc1[0]-scaleFac),(ptLoc1[1]+scaleFac,ptLoc1[0]+scaleFac),255*color[i],2,cv2.LINE_AA)
            cv2.rectangle(outImg,(ptLoc2Shifted[1]-scaleFac,ptLoc2Shifted[0]-scaleFac),(ptLoc2Shifted[1]+scaleFac,ptLoc2Shifted[0]+scaleFac),255*color[i],2,cv2.LINE_AA)            

        cv2.line(outImg,(ptLoc1[1],ptLoc1[0]),(ptLoc2Shifted[1],ptLoc2Shifted[0]),255*color[i],2,cv2.LINE_AA)

    if show:
        plt.imshow(outImg)
        plt.show()
    return outImg


def getFeatureCorresConventional(im1_,im2_):

    orb = cv2.ORB_create()#(nfeatures=5000,scaleFactor=1.2,nlevels=5,edgeThreshold=1,patchSize=20)
#     sift = cv2.ORB_create(nfeatures=5000)#,edgeThreshold=1,patchSize=20)
    
    # find the keypoints and descriptors with SIFT
    kp1, des1 = orb.detectAndCompute(im1_,None)
    kp2, des2 = orb.detectAndCompute(im2_,None)

    
#     # FLANN parameters
#     FLANN_INDEX_KDTREE = 0
#     index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#     search_params = dict(checks=50)
    
    flann = cv2.BFMatcher()#index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    
    good = []
    pts1 = []
    pts2 = []
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt) 
            
    pts1 = np.array(pts1)
    pts2 = np.array(pts2)

    showImg = None
    plt.figure(figsize=(15,10))
    showImg=getCorresImg(im1,im2,list(zip(np.fliplr(pts1),np.fliplr(pts2))),True,1)    
    plt.show()
    
    return pts1, pts2, showImg
    
def trackPair(binImg1,binImg2,rawImg1,rawImg2):
    
    # get pose between raw images # (can also try embedding images for this purpose for robustness)
    
    # get pixels to track from the binary images
    
    # project pixels to the other image using the pose
    
    # display
    return

In [ ]:

im1 = loadRawImg(rawImgPath,0)
im2 = loadRawImg(rawImgPath,1)
pts1, pts2, _ = getFeatureCorresConventional(im1,im2)

### Essential Matrix (Epipolar Geometry) 

In [ ]:
# E,mask = cv2.findEssentialMat(pts2, pts1,)# focal, pp, RANSAC, 0.999, 1.0, mask);
# numPoints,R,t,mask = cv2.recoverPose(E, pts2, pts1,)# R, t, focal, pp, mask);
E,mask = cv2.findEssentialMat(pts2, pts1, f0, (xx0,yy0), cv2.RANSAC, 0.999, 1.0)
numPoints,R,t,mask = cv2.recoverPose(E, pts2, pts1, focal=f0, pp=(xx0,yy0),mask=mask);
print(numPoints,R,t,mask.sum())
mask = mask.astype(bool)
def projPoints(pts,R_,t_):
    ptsH = cv2.convertPointsToHomogeneous(pts)[:,0,:]
    return cv2.convertPointsFromHomogeneous(np.matmul(ptsH,R_) + t_.transpose())[:,0,:]
pts1_ = projPoints(pts1,R,t)
print(pts1_.shape)
plt.figure(figsize=(15,10))
showImg=getCorresImg(im1,im2,list(zip(np.fliplr(pts1[mask[:,0]]),np.fliplr(pts1_[mask[:,0]]))),True,1)    
plt.show()

In [ ]:
projPoints(np.array([[550,375],[600,425]]),R,t)

### Homography 

In [ ]:
M, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)
matchesMask = mask.ravel().tolist()

plt.figure(figsize=(15,10))
showImg=getCorresImg(im1,im2,list(zip(np.fliplr(pts1[mask[:,0]]),np.fliplr(pts2[mask[:,0]]))),True,1)    
plt.show()

h,w,_ = im1.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0],[550,375],[600,425] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,M)

img2 = cv2.polylines(im2.copy(),[np.int32(dst)],True,255,3, cv2.LINE_AA)
plt.imshow(img2)

### Image Alignment 

In [ ]:
_, W = cv2.findTransformECC(cv2.cvtColor(im1[300:500],cv2.COLOR_BGR2GRAY),
                     cv2.cvtColor(im2[300:500],cv2.COLOR_BGR2GRAY),
                     np.eye(3,dtype=np.float32),motionType=cv2.MOTION_HOMOGRAPHY)

In [ ]:
h,w,_ = im1.shape
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0],[550,75],[600,125] ]).reshape(-1,1,2)
dst = np.matmul(W,cv2.convertPointsToHomogeneous(pts[:,0,:])[:,0,:].transpose())[:2,:].transpose()#cv2.perspectiveTransform(pts,W)
dst = np.matmul(W,np.array([[550,75,1],[600,125,1]]).transpose()).transpose()[:2,:2]
img1 = cv2.polylines(im1[300:500].copy(),[np.int32([[550,75],[600,125]])],True,255,3, cv2.LINE_AA)
plt.imshow(img1)
plt.show()
img2 = cv2.polylines(im2[300:500].copy(),[np.int32(dst)],True,255,3, cv2.LINE_AA)
plt.imshow(img2)

### Optical Flow 

In [ ]:
pts2_,stat,err = cv2.calcOpticalFlowPyrLK(im1,im2,lanePts1.astype(np.float32),None)
plt.figure(figsize=(15,10))
showImg=getCorresImg(im1,im2,list(zip(lanePts1,pts2_)),True,1)    
plt.show()
# print(stat,err)

#### Dense 

In [ ]:
flowImg = cv2.calcOpticalFlowFarneback(
#     cv2.cvtColor(im1[300:500],cv2.COLOR_BGR2GRAY),
#     cv2.cvtColor(im2[300:500],cv2.COLOR_BGR2GRAY),
    cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY),
    cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY),    
    None,0.5,3,31,100,5,1.1,cv2.OPTFLOW_FARNEBACK_GAUSSIAN)

In [ ]:
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = img.copy()#cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [ ]:
# vis = draw_flow(im1[300:500],flowImg)
vis = draw_flow(im1,flowImg)
plt.figure(figsize=(15,10))
plt.imshow(vis)
plt.show()

In [ ]:
%matplotlib inline
lp_flow = flowImg[lanePts1[:,0]-300,lanePts1[:,1]]
pts2_ = lanePts1-[300,0] + np.fliplr(lp_flow)
plt.figure(figsize=(15,10))
showImg=getCorresImg(im1[300:500],im2[300:500],list(zip(lanePts1-[300,0],pts2_)),True,1)    
plt.show()

### Embeddings thresholding 

In [ ]:
%matplotlib inline
_, embImg = loadProcImg(dataLoadPath,2,True)
embImg[:,:,2] = 0
embImg[embImg<embImg.mean()+2*embImg.std()] = 0
plt.imshow(embImg)
plt.colorbar()
embImg.mean()

### Load point labels

In [ ]:
ptLabels = np.load(dataLoadPath+"0000002.jpg.npz")['arr_0']
print(ptLabels.max())
plt.imshow(ptLabels)


#### Temp Misc 

In [ ]:
im1 = cv2.imread("../out/filtered-loop/cfdn-f11/11.jpg")
im2 = cv2.imread("../out/filtered-loop/cfdn-f11/12.jpg")
im3 = cv2.imread("../out/filtered-loop/cfdn-f11/13.jpg")

plt.figure(figsize=(30,10))
plt.imshow(np.hstack([im1,im2,im3]))
plt.tight_layout()
# plt.savefig("./trackedPoints.jpg")

### Load deep dense optical flow 

In [ ]:
idx = 49
# flo = np.load("../../UnFlow/out/CSS/"+"{0:06d}".format(idx)+"_flow.png.npz")['arr_0'][0]
flo = cv2.imread("../../UnFlow/out/CSS/"+"{0:06d}".format(idx)+"_flow.png",cv2.IMREAD_UNCHANGED)[:,:,:2].astype(float)-500
# flo = np.loadtxt("../../flownet2-pytorch/work/inference/run.epoch-0-flow-field/000000.flo")
print(flo.shape)

In [ ]:
vis = draw_flow(loadRawImg(rawImgPath,idx),flo)
plt.figure(figsize=(15,10))
plt.imshow(vis)
plt.show()

plt.imshow(flo[:,:,0])
plt.colorbar()
plt.show()

plt.imshow(flo[:,:,1])
plt.colorbar()
plt.show()


### Post process tracked points 

In [ ]:
import sys
sys.path.append('/home/localuser/workspace/lanenet-lane-detection/')

from lanenet_model import lanenet_postprocess
postprocessor = lanenet_postprocess.LaneNetPoseProcessor()


In [ ]:
idx = 52
img1 = loadRawImg(rawImgPath,idx)
prevPoints = getProjectedPoints(idx,idx+1)
print(prevPoints.shape) 

binary_seg_image = np.zeros(img1.shape,np.uint8)
binary_seg_image[prevPoints[:,0],prevPoints[:,1],:] = (255,255,255)
plt.imshow(binary_seg_image)
plt.show()

procImg = postprocessor.postprocess(binary_seg_image)
plt.imshow(procImg)
plt.show()

In [ ]:
out = label(procImg)
plt.imshow(255*out/float(out.max()))

regions = regionprops(out)
fitImg = np.zeros(img1.shape,np.uint8)
# plt.imshow(fitImg)

for i1 in range(len(regions)):
    cords = regions[i1].coords
    y = cords[:,0]
    x = cords[:,1]
    
    fit = np.polyfit(x,y,1)
    fit_fn = np.poly1d(fit) 
    
    xnew = np.arange(x.min(),x.max())
    ynew = fit_fn(xnew).astype(int)
        
    fitImg[ynew,xnew,:] = (255,255,255)

    fitImg = cv2.line(fitImg,(xnew[0],int(fit_fn(xnew[0]))),(xnew[-1],int(fit_fn(xnew[-1]))),
                      (255,255,255),2,cv2.LINE_AA)

#     plt.plot(xnew,ynew)
plt.imshow(fitImg)
# cv2.imwrite("../tmp.jpg",fitImg)


### Get label from color 

In [ ]:
colorsDef = np.array([np.array([255, 0, 0]),
                           np.array([0, 255, 0]),
                           np.array([0, 0, 255]),
                           np.array([125, 125, 0]),
                           np.array([0, 125, 125]),
                           np.array([125, 0, 125]),
                           np.array([50, 100, 50]),
                           np.array([100, 50, 100])])
colorCodeUni = np.sum(colorsDef*[100,10,1],axis=1)
print(colorCodeUni)
assert(len(colorCodeUni) == len(np.unique(colorCodeUni)))

codeConverter = np.zeros(np.max(colorCodeUni)*10,int)
codeConverter[colorCodeUni] = np.arange(1,len(colorCodeUni)+1)
print(np.unique(codeConverter,True))

In [ ]:
# %matplotlib inline
_,_,imColLab = loadProcImg(dataLoadPath,6,True,True) 
plt.imshow(imColLab)
plt.show()
# print(np.unique(imColLab))

# imColLab_Uni = np.sum(imColLab*np.array([100,10,1])[None,None,:],axis=2)
# plt.imshow(imColLab_Uni)
# plt.colorbar()
# plt.show()
# print(np.unique(imColLab_Uni).shape)

# imLabel = codeConverter[imColLab_Uni]
# plt.imshow(imLabel*100)
# plt.colorbar()
# plt.show()
# print(np.unique(imLabel))

#### From embeddings image 

In [ ]:
%matplotlib inline
colorsDef = np.array([np.array([255, 0, 0]),
                           np.array([0, 255, 0]),
                           np.array([0, 0, 255]),
                           np.array([255, 255, 0]),
                           np.array([0, 255, 255]),
                           np.array([255, 0, 255]),
                           np.array([255, 255, 255]),
                           np.array([0, 0, 0])])

# _,imEmb = loadProcImg(dataLoadPath,6,True,False) 
# plt.imshow(imEmb)
# plt.show()

im2 = np.argmin(np.sum(abs(imEmb[None] - colorsDef[:,None,None,:]),axis=-1),axis=0).astype(float)
# im2[im2!=7] = 7
plt.imshow(im2)
plt.colorbar()
plt.show()
